In [5]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import boto3

In [6]:
host = 'search-photos-fi6tkdtpua4mm3lpfuebg26sdi.us-east-1.es.amazonaws.com'
region = 'us-east-1'
botID = '0YKQS6ZTGQ'
botAliasID = 'CJXWCG6WUW'
sessionID = 'session'
localeID = 'en_US'

credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region)
index_name = 'photo'
opensearch = OpenSearch(
    hosts = [{'host': host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)
lex = boto3.client('lexv2-runtime')

In [8]:
opensearch.indices.create(index_name)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'photo'}

In [39]:
def get_s3_paths(labels):
    query = {
              'size': 5,
              'query': {
                'multi_match': {
                  'query': "+".join(labels),
                  'fields': ["label"]
                }
              }
            }
    response = opensearch.search(
        body = query,
        index = index_name
    )
    return [hits['_source']['s3_path'] for hits in response['hits']['hits']]

In [48]:
query = {
          'size': 5,
          'query': {
            'match': {
              'query': "hello+world",
              'fields': ["label"]
            }
          }
        }
response = opensearch.search(
    body = query,
    index = index_name
)
[hits['_source']['s3_path'] for hits in response['hits']['hits']]

['https://bucket-album.s3.amazonaws.com/kudasai.jpg',
 'https://bucket-album.s3.amazonaws.com/kudukk.jpg',
 'https://bucket-album.s3.amazonaws.com/kudukk.jpg',
 'https://bucket-album.s3.amazonaws.com/kudasai.jpg']

In [41]:
get_s3_paths(['hello', 'world'])

['https://bucket-album.s3.amazonaws.com/kudasai.jpg',
 'https://bucket-album.s3.amazonaws.com/kudukk.jpg',
 'https://bucket-album.s3.amazonaws.com/kudukk.jpg',
 'https://bucket-album.s3.amazonaws.com/kudasai.jpg']

In [21]:
def get_labels(s3_path):
    query = {
              'size': 20,
              'query': {
                'multi_match': {
                  'query': s3_path,
                  'fields': ["s3_path"]
                }
              }
            }
    response = opensearch.search(
        body = query,
        index = index_name
    )
    return [hits['_source']['label'] for hits in response['hits']['hits']]

In [27]:
def lambda_handler(event, context):
    full_query = event['q']
    response = lex.recognize_text(
            botId=botID,
            botAliasId=botAliasID,
            localeId=localeID,
            sessionId=sessionID,
            text=full_query)

    result = {'result': []}
    if response['sessionState']['intent']['name'] in ['SearchIntent', 'FallbackIntent']:
        queries = full_query.split(' ')
        images = set()
        for query in queries:
            query = ''.join(x for x in query if x.isalpha()).lower()
            s3_paths = get_s3_paths(query)
            images.update(s3_paths)
        images = list(images)

        for image in images:
            result['result'].append(
                {
                    'url': image,
                    'labels': get_labels(image)
                }
            )
    return result 

In [28]:
def lambda_handler(event, context):
    query = event['q']
    
    response = client.recognize_text(
            botId=botID,
            botAliasId=botAliasID,
            localeId=localeID,
            sessionId=sessionID,
            text=query)
    
    outgoing_msgs = response['messages']
    session_intent = response['sessionState']['intent']
    
    if (session_intent['name'] == 'DiningSuggestionsIntent') & (session_intent['state'] == 'ReadyForFulfillment') & (session_intent['confirmationState'] == 'Confirmed'):
        print({k: v['value']['interpretedValue'] for k, v in slots.items()})
    
    if len(outgoing_msgs):
        msgs = []
        for outgoing_msg in outgoing_msgs:
            print("outgoing_msg : {}".format(outgoing_msg))
            outgoing_msg_content = outgoing_msg['content']
            msgs.append({
                    "type": "unstructured",
                    "unstructured": {
                    "id": str(uuid.uuid4()),
                    "text": outgoing_msg_content,
                    "timestamp": datetime.datetime.isoformat(datetime.datetime.now()),
                    }
                })
        return {
                "messages": msgs
            }
    else:
        raise ValueError("Lex has Failed")

{'result': [{'url': 'https://bucket-album.s3.amazonaws.com/kudasai.jpg',
   'labels': ['hello',
    'Abyssinian',
    'Cat',
    'Chair',
    'world',
    'Pet',
    'Mammal',
    'Cushion',
    'Couch',
    'Animal',
    'Kitten',
    'Manx',
    'Couch',
    'Kitten',
    'cuki',
    'Manx',
    'cuki',
    'Chair',
    'Cat',
    'Pet']},
  {'url': 'https://bucket-album.s3.amazonaws.com/kudukk.jpg',
   'labels': ['Manx',
    'Chair',
    'Couch',
    'Kitten',
    'hello',
    'world',
    'Abyssinian',
    'Cat',
    'Pet',
    'Animal',
    'Mammal',
    'Cushion',
    'Cat',
    'Chair',
    'cuki',
    'world',
    'Pet',
    'cuki',
    'Mammal',
    'Cushion']}]}

In [23]:
get_labels('https://bucket-album.s3.amazonaws.com/kudasai.jpg')

['hello',
 'Abyssinian',
 'Cat',
 'Chair',
 'world',
 'Pet',
 'Mammal',
 'Cushion',
 'Couch',
 'Animal',
 'Kitten',
 'Manx',
 'Couch',
 'Kitten',
 'cuki',
 'Manx',
 'cuki',
 'Chair',
 'Cat',
 'Pet']